In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
# To get learning rate


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

# X = pd.concat([train.drop(['click'],axis=1),test])
X = pd.concat([train,test],sort=False)
print(X.shape)

X_clean = pd.read_csv('../../data/original/cleaned_data_final.csv')

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
print(X.shape)
assert np.sum(X_clean['instance_id'].values != X['instance_id'].values) == 0
X['model'] = X_clean['model_new'].values
# X['make'] = X_clean['make_new'].values

ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 35)
(1041674, 45)
(1041674, 36)


In [3]:
X.columns

Index(['instance_id', 'time', 'city', 'province', 'user_tags', 'carrier',
       'devtype', 'make', 'model', 'nnt', 'os', 'osv', 'os_name', 'adid',
       'advert_id', 'orderid', 'advert_industry_inner', 'campaign_id',
       'creative_id', 'creative_tp_dnf', 'app_cate_id', 'f_channel', 'app_id',
       'inner_slot_id', 'creative_type', 'creative_width', 'creative_height',
       'creative_is_jump', 'creative_is_download', 'creative_is_js',
       'creative_is_voicead', 'creative_has_deeplink', 'app_paid',
       'advert_name', 'click', 'time_hour'],
      dtype='object')

In [4]:
# def comb_fe(X,cols,sep=' '):
#     ret = X[cols[0]].astype(str).copy()
#     for col in cols[1:]:
#         ret = ret + sep + X[col].astype(str)
#     return ret.values

# processed_col = []
# cob_col = [['model','make','os','osv']]
# doc_col=['user_tags']
# X_doc = X[doc_col].copy()
# processed_col.extend(doc_col)
# for each in cob_col:
#     feature_name = '_'.join(each)
#     processed_col.extend(each)
#     doc_col.append(feature_name)
#     X_doc[feature_name] = comb_fe(X,each)
    
# non_doc_col = list(set(X.columns) - set(ignore_columns) - set(processed_col))
# X_ = X[non_doc_col].copy()

# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     X_doc[col] = X_doc[col].astype(str)

In [5]:
# # def comb_fe(X,cols,sep=' '):
# #     ret = X[cols[0]].astype(str).copy()
# #     for col in cols[1:]:
# #         ret = ret + sep + X[col].astype(str)
# #     return ret.values

# def process_slot(x):
#     x = re.sub(r'[-_]',' ',x)
#     x = x.split(' ')
#     ret = ['p{}_'.format(c)+x[c] for c in range(len(x))]
#     ret = ' '.join(ret)
#     return ret
        
    

# def comb_fe(X,cols,sep=' '):
#     def add_col_name(x,colName,splitter=' ' ,sep=' '):
#         ret = [colName+'_'+each for each in x.split(splitter)]
#         return sep.join(ret)
#     ret = pd.DataFrame()
#     if cols[0] == 'user_tags':
#         ret['comb'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#     else:
#         if cols[0] in original_name_col:
#             ret['comb'] = X[cols[0]].astype(str).copy()
#         else:
#             ret['comb'] = X[cols[0]].astype(str).apply(add_col_name,colName=cols[0],splitter=' ',sep=sep)

#     for col in tqdm(cols[1:]):
#         if col == 'user_tags':
#             ret['new_feature'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#         else:
#             if col in original_name_col:
#                 ret['new_feature'] = X[col].astype(str).copy()
#             else:
#                 ret['new_feature'] = X[col].astype(str).apply(add_col_name,colName=col,splitter=' ',sep=sep)
#         ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
#     return ret['comb'].values

# processed_col = []
# original_name_col = ['model','make','os','osv']
# #41481
# # cob_col = [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']]
# #4179
# # cob_col = [['model','osv']]
# #41479 with duplicate in non-doc
# # cob_col = [['model','make','os','osv']]
# #41480 + inner slot id
# # cob_col = [['model','make','os','osv']]
# #41480
# doc_col=['user_tags','inner_slot_id']
# X_doc = X[doc_col].copy()
# processed_col.extend(doc_col)
# for each in cob_col:
#     feature_name = '_'.join(each)
#     processed_col.extend(each)
#     doc_col.append(feature_name)
#     X_doc[feature_name] = comb_fe(X,each)
    
# non_doc_col = list(set(X.columns) - set(ignore_columns) - set(processed_col))
# #!!!!!!!!!!!!!!!!!!!!!!!!
# # 0.414797
# # non_doc_col = list(set(X.columns) - set(ignore_columns)- set(doc_col))
# X_ = X[non_doc_col].copy()

# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     if col=='inner_slot_id':
#         X_doc[col] = X_doc[col].astype(str).apply(process_slot)
#     else:
#         X_doc[col] = X_doc[col].astype(str)
    
        

100%|██████████| 3/3 [00:03<00:00,  1.16s/it]


# rnn all except user_tags

In [6]:
# cobdoc = list(set(X.columns) - set(ignore_columns)-set(['user_tags']))
# doc_col = ['user_tags']
# non_doc_col = []
# def comb_fe(X,cols,sep=' '):
#     ret = pd.DataFrame()
#     le = LabelEncoder()
#     ret['new_feature'] = le.fit_transform(X[cols[0]].astype(str))
#     ret['new_feature'] = cols[0]+'_'+ ret['new_feature'].astype(str)
#     ret['comb'] = ret['new_feature'].values

#     for col in tqdm(cols[1:]):
#         ret['new_feature'] = le.fit_transform(X[col].astype(str))
#         ret['new_feature'] = col+'_'+ ret['new_feature'].astype(str)
#         ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
#     return ret['comb'].values
# X_doc = X[doc_col].copy()
# X_doc['combinesALL'] = comb_fe(X,cobdoc)
# doc_col.append('combinesALL')

# for col in tqdm(doc_col):
#     X_doc[col] = X_doc[col].astype(str)

# rnn all including user tags

In [7]:
# def comb_fe(X,cols,sep=' '):
#     ret = pd.DataFrame()
#     if cols[0] == 'user_tags':
#         ret['comb'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#     else:
#         ret['comb'] = X[cols[0]].astype(str).apply(add_col_name,colName=cols[0],splitter=' ',sep=sep)

#     for col in tqdm(cols[1:]):
#         if col == 'user_tags':
#             ret['new_feature'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#         else:
#             ret['new_feature'] = X[col].astype(str).apply(add_col_name,colName=col,splitter=' ',sep=sep)
#         ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
#     return ret['comb'].values


# def add_col_name(x,colName,splitter=' ' ,sep=' '):
#     ret = [colName+'_'+each for each in x.split(splitter)]
#     return sep.join(ret)

# non_doc_col = []
# needsProcessCols = list(set(X.columns) - set(ignore_columns))
# doc_col = ['combineALL']
# X_doc = pd.DataFrame()
# X_doc['combineALL'] = comb_fe(X,needsProcessCols,sep=' ')
    

In [8]:
need_process_col = list(set(X.columns) - set(ignore_columns))
X_ = X[need_process_col].copy()


doc_col = ['user_tags','model']
non_doc_col = [f for f in need_process_col if f not in doc_col]
# doc_col = doc_col + []
# non_doc_col = non_doc_col+['user_tags']
counter = 0

X_doc = X[doc_col].copy()
for col in tqdm(non_doc_col):
    X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = col + '_'+X_[col].astype(str)
    
for col in tqdm(doc_col):
    X_doc[col] = X_doc[col].astype(str)
    
    

    

In [9]:
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

In [10]:
num_folds = 7
seed = 1001
train_index_list = []
val_index_list = []
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
for t,v in folds.split(train.loc[train_index],train.loc[train_index,'click']):
    train_index_list.append(train.loc[train_index].iloc[t].index)
    val_index_list.append(train.loc[train_index].iloc[v].index)
    
check = []
for i in val_index_list:
    check.extend(list(i))
assert len(set(check+list(holdout_index))) == len(train)

In [11]:
train_fold_y = {}
val_fold_y = {}
holdout_y = train.loc[holdout_index,'click'].values
for fold in range(num_folds):
    train_fold_y[fold] = train.loc[train_index_list[fold],'click'].values
    val_fold_y[fold] = train.loc[val_index_list[fold],'click'].values

In [12]:
info_dict = {}
train_all_dict = {}
train_fold_dict = {}

val_fold_dict = {}

holdout_input_dict = {}
test_input_dict = {}
maxlen = 1
prefix_input_nonDoc = 'input_'
prefix_input_Doc = 'input_rnn_'
for col in tqdm(non_doc_col):

    maxlen = 1
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='@')
    tok.fit_on_texts(list(X_[col]))
    info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
    t = tok.texts_to_sequences(list(X_[col].iloc[:1001650].values))
    te = tok.texts_to_sequences(list(X_[col].iloc[1001650:].values))
    train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    holdout_input_dict[prefix_input_nonDoc+col] = train_all_dict[prefix_input_nonDoc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})
        
sequence_size_dict = {}
for col in tqdm(doc_col):
    if col == 'user_tags':
        maxlen = 50
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters=',')
    else:
        maxlen = 20
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters=' ')
    tok.fit_on_texts(list(X_doc[col]))
    info_dict.update({prefix_input_Doc+col:{'tok':tok}})
    sequence_size_dict[col] = maxlen
    t = tok.texts_to_sequences(list(X_doc[col].iloc[:1001650].values))
    te = tok.texts_to_sequences(list(X_doc[col].iloc[1001650:].values))
    train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_Doc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    holdout_input_dict[prefix_input_Doc+col] = train_all_dict[prefix_input_Doc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})

100%|██████████| 3/3 [00:56<00:00, 18.79s/it]


In [13]:
# tok = info_dict['input_rnn_inner_slot_id']['tok']
# tok.word_index

# Build NN model only use model

# 0.41463 -- 71 epoch

In [14]:

def get_nn_model(cols,doc_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
        x = SpatialDropout1D(0.5)(embed_layer)
        x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
        x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
        x_aveP = GlobalAveragePooling1D()(x)
        x_maxP = GlobalMaxPooling1D()(x)
        x = concatenate([x_aveP, x_maxP]) 
        input_list.append(cur_input)
        concat_list.append(x)
#         concat_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
#     x = Dense(512)(x)

#     x = Activation('relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

In [15]:

# def get_nn_model(cols,doc_cols=[]):
#     input_list = []
#     concat_list = []
#     reg = 0.0005
#     for col in cols:
#         max_feature = len(info_dict[col]['tok'].index_word)
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         if col in doc_cols:
#             cur_input = Input(shape=(50, ),name = 'input_'+col)
#             embed_layer = Embedding(max_feature,
#                                 embed_size,
#                                 input_length=50,
#                                 trainable=True,
#                                 embeddings_regularizer=l2(reg),
#                                 name='ebd_'+col)(cur_input)
#             x = SpatialDropout1D(0.5)(embed_layer)
#             x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
#             x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#             x_aveP = GlobalAveragePooling1D()(x)
#             x_maxP = GlobalMaxPooling1D()(x)
#             x = concatenate([x_aveP, x_maxP]) 
# #             x = Flatten()(x)
#             concat_list.append(x)
            
#         else:
#             cur_input = Input(shape=(1, ),name = 'input_'+col)
        
       
#             embed_layer = Embedding(max_feature,
#                                 embed_size,
#                                 input_length=1,
#                                 trainable=True,
#                                 embeddings_regularizer=l2(reg),
#                                 name='ebd_'+col)(cur_input)
#             embed_layer = SpatialDropout1D(0.5)(embed_layer)
# #             x = Conv1D(embed_size*2, kernel_size = 1, padding = "valid", kernel_initializer = "glorot_uniform")(embed_layer)
# #             x_aveP = GlobalAveragePooling1D()(x)
# #             x_maxP = GlobalMaxPooling1D()(x)
# #             x = concatenate([x_aveP, x_maxP]) 
#             x = Flatten()(embed_layer)
#         input_list.append(cur_input)
#         concat_list.append(x)
    
#     x = concatenate(concat_list)
# #     x = BatchNormalization()(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.2)(x)
# #     x = Dense(512)(x)

# #     x = Activation('relu')(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model(input_list, preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
#     return model

In [16]:
def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],tolerance=30,):
    model = get_nn_model(cols,doc_col)
    cur_to = 0
    best_logloss = None
    best_weights = None
    count = 0
    base_lr = 0.001
    while True:
#         decay = (80 - count)/80
#         decay = max(decay,0.1)
#         lr = base_lr * decay
#         K.set_value(model.optimizer.lr, lr)
        model.fit(input_train_dict, y_train, 
                  batch_size=5000, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,5000,verbose=1)
        logloss = log_loss(y_val,preds)
        roc = roc_auc_score(y_val,preds)
        print(logloss)
        print(roc)
        if best_logloss is None:
            best_logloss = logloss
            best_weights = model.get_weights()
        else:
            if best_logloss > logloss:
                best_logloss = logloss
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best logloss is: {}'.format(best_logloss))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    return model
    

In [17]:
# model = train_each_fold(t_notag,v_notag,train_fold_y[0],val_fold_y[0],col_notag,doc_col)

In [18]:
model = train_each_fold(train_fold_dict[0],val_fold_dict[0],train_fold_y[0],val_fold_y[0],non_doc_col,doc_col)

Epoch 1/1
137376/137376 [==============================] - 2s 12us/step
0.4206240229920487
0.7599462012943433
best logloss is: 0.4206240229920487
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.41803811494233883
0.7632496483493093
best logloss is: 0.41803811494233883
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.418018104771168
0.7636277416358157
best logloss is: 0.418018104771168
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.4175653872157127
0.7643206129819407
best logloss is: 0.4175653872157127
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
137376/137376 [==============================] - 1s 9us/step
0.4183466879432787
0.7642007461661436
best logloss is: 0.4175653872157127
remainning trial is: 1/30
total epoch trained: 4
Epoch 1/1
137376/137376 [=

137376/137376 [==============================] - 1s 10us/step
0.4154537005621384
0.7688781831850748
best logloss is: 0.41495911786984535
remainning trial is: 5/30
total epoch trained: 57
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.41532855017201914
0.769000601611874
best logloss is: 0.41495911786984535
remainning trial is: 6/30
total epoch trained: 58
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.4150700882275774
0.76964776121065
best logloss is: 0.41495911786984535
remainning trial is: 7/30
total epoch trained: 59
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.41536063075953095
0.7689415232739847
best logloss is: 0.41495911786984535
remainning trial is: 8/30
total epoch trained: 60
Epoch 1/1
137376/137376 [==============================] - 1s 10us/step
0.41498214417477125
0.7695221838053332
best logloss is: 0.41495911786984535
remainning trial is: 9/30
total epoch trained: 61
Epoch 1/1
137376/137376 [

KeyboardInterrupt: 

In [ ]:
model.optimizer.lr.

In [ ]:
input_val_dict = val_fold_dict[0]
y_val = val_fold_y[0]
preds = model.predict(input_val_dict,5000,verbose=1)
logloss = log_loss(y_val,preds)
roc = roc_auc_score(y_val,preds)
print(logloss)
print(roc)

In [ ]:
# predsub = model.predict(test_input_dict,5000,verbose=1)
# test_save = test[['instance_id']].copy()
# test_save['predicted_score'] = predsub

In [ ]:
# test_save.to_csv('ebd_nn.csv',index=False)


In [ ]:
model = get_nn_model(need_process_col,doc_col)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')